In [1]:
import json
import random
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.agents.math_qa_correction_agent import MathQACorrectionAgent
from src.utils.data_generation import split_dataset, concurrent_data_postprocessing
random.seed(2)

In [2]:
PROC_NUM = 5
ERROR_DETECTION_ONLY = False
SPLIT = "train"

In [3]:
with open(f"../../../dataset/raw_model_responses/{SPLIT}/{SPLIT}_UMWP.json", "r") as f:
    json_data = json.load(f)

In [4]:
json_data = json_data[:20]

In [5]:
data_chunks = split_dataset(json_data, PROC_NUM)

In [6]:
agent = MathQACorrectionAgent(error_detection_only=ERROR_DETECTION_ONLY)

In [7]:
def extract_args(item: dict) -> list[str]:
    return {
        "input": item["input"],
        "question": item["additional_info"]["question"],
        "answer": item["additional_info"]["answer"],
        "is_answerable": item["additional_info"]["answerable"],
        "responses": item["response"],
    }

In [8]:
data = await concurrent_data_postprocessing(agent, data_chunks, extract_args)

Response len: 10Response len: 10

Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10
Response len: 10


2025-08-02 19:22:42.563 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:104 - Errors: [[], [], [], [], [], [], [], [], [], []]
2025-08-02 19:22:42.564 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:105 - Wrong responses: 0


Response len: 10


2025-08-02 19:22:43.082 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:104 - Errors: [[], [], [], [], [], [], [], [], [], []]
2025-08-02 19:22:43.083 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:105 - Wrong responses: 0


Response len: 10


2025-08-02 19:22:43.435 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:104 - Errors: [[], [], [], [], [], [], [], [], [], []]
2025-08-02 19:22:43.436 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:105 - Wrong responses: 0


Response len: 10


2025-08-02 19:22:44.007 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:104 - Errors: [[], [], [], [], [], [{'description': "The problem states that 35 people didn't show up, so the total number of people who attended the luncheon is not 45. It is 45 - 35 = 10.", 'location': 'The total number of people who attended the luncheon is 45.', 'correction': 'DELETE: "The total number of people who attended the luncheon is 45."\nADD: "The total number of people who attended the luncheon is 45 - 35 = 10."'}, {'description': 'The calculation uses the wrong number of people. It should be 10 / 2 = 5, not 45 / 2 = 22.5', 'location': 'To find the number of tables needed, divide 45 (total number of people) by 2 (number of people each table can hold). tables_needed = 45 / 2 = 22.5', 'correction': 'DELETE: "45 (total number of people) by 2 (number of people each table can hold). tables_needed = 45 / 2 = 22.5"\nADD: "10 (total number of people) by 2 (number of people each table can hol

Response len: 10


2025-08-02 19:22:44.536 | INFO     | src.agents.math_qa_correction_agent:check_for_errors:104 - Errors: [[{'description': 'The subtraction is performed in the wrong order. The problem asks how many *more* students suggested bacon, so we should subtract the number of mashed potato suggestions from the number of bacon suggestions, not the other way around.', 'location': '479 - 489 = -10', 'correction': 'DELETE: "479 - 489 = -10"\nADD: "489 - 479 = 10"'}, {'description': 'The interpretation of the result is incorrect because the subtraction was done in the wrong order. A negative result would indicate that fewer students suggested bacon, but the calculation should have been 489 - 479 = 10, indicating that 10 more students suggested bacon.', 'location': 'Since the result of the subtraction is negative, it means that there were more students who suggested mashed potatoes than those who suggested bacon.', 'correction': 'DELETE: "Since the result of the subtraction is negative, it means that 

In [9]:
data[4]

{'input': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a meticulous AI mathematician. Your task is to solve the following math problem.\nThink step by step. Show all your reasoning and calculations. After you have solved the problem, clearly state the final numerical answer at the end of your reasoning.\nIf the question is unanswerable (e.g., it is illogical or missing information), you must clearly state that it cannot be answered and briefly explain why. Do not attempt to solve it.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nLuke was helping the cafeteria workers pick up lunch trays, but he could only carry -1 trays at a time. If he had to pick up 20 trays from one table and 16 trays from another, how many trips will he make? <|eot_id|><|start_header_id|>assistant<|end_header_id|>',
 'question': 'Luke was helping the cafeteria workers pick up lunch trays, but he could only carry -1 trays at a time. If he had to pick up 20 trays from one table and 1

In [ ]:
with open(f"../../../dataset/processed_data/{SPLIT}/UMWP_processed.json", "w") as f:
    json.dump(data, f, indent=4)